<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/250103_%EA%B0%9C%EB%B0%9C%EC%9D%BC%EC%A7%80_Chunk_Unicode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# unicode
- 간단히 string 비교에 문제 있는 것 기록하고 넘어가기
``` python
if "20171004(수요_사31장)" in txt_file:
    print("Condition satisfied")
```
- 멀쩡히 있는 파일인데 if 문 안으로 안들어가길래 이상하다 싶어서 숫자만 남겼더니 조건문 안으로 들어감
- unicode
``` python
normalized_name = unicodedata.normalize('NFC', txt_file)
if "20171004(수요_사31장).txt" in normalized_name:
    print("Condition satisfied")
```
- 한글을 표시하는 unicode encode에 문제가 있었던 것 같다. 참고로 특수문자 '('는 문제 없음
- 사실 지금 쓰는 markup도 한글 입력이 잘 안되는 경우가 많다.
    
# 최초 구상했던 chunk 방식
- chunk를 우선 잘 나누는 것 부터 해보자.
- 지금까지 사용하지 않았던 설교 요약집이 있다.
- 요즘은 사진에서 한글 text 긁어오는 기술도 굉장히 발달해서 거의 오류 없이 가져오는 것이 가능했다. from google photo
- 이 설교 요약집을 label 자료 혹은 chunk 단위로 사용할 수 있다.
- 크게는 4~5개 단락으로 나눈다.
    - 서론, 첫 번째, 두 번째, 세 번째, (네 번째)
    - 그리고 각 단락 안에서 어떤 의미 집합으로 chunk를 구성때지 정할 때 요약본을 사용하면 된다.
- 그리고 내가 만드는 LLM은 가장 유사도가 높은 chunk를 찾는 것 부터 시작해야겠다.

# chunk
- 사용자가 질문을 하면 model은 가장 유사도 높은 내용을 검색해서 제공하면 되지 않나? 모델은 왜 필요하지?
    > ChatGPT가 답변을 생성하는 과정은 **찾기(Search)**와 **생성(Generate)**의 혼합에 가깝습니다. 엄밀히 따지면 찾기보다는 패턴에 기반한 생성에 더 가깝습니다.     
    > 1. 찾기(Search)의 성격: 학습된 데이터(인터넷에서 수집된 정보 포함)를 기반으로 학습되었기 때문에 기존 정보에서 패턴과 맥락을 이해하고 이를 활용합니다. 그러나 실시간 검색을 통해 정보를 가져오는 것은 아니며, 학습된 데이터에서 지식을 "검색"한다고 볼 수는 있습니다.  
    > 2. 생성(Generate)의 성격: GPT 모델은 학습한 데이터로부터 확률적으로 단어와 문장을 생성합니다. 단순히 데이터를 복사하거나 기존 지식을 나열하는 것이 아니라, 새로운 맥락에서 재구성하거나 유추를 통해 답변을 만듭니다.   
    > 3. 결론적으로: "찾는다"는 표현은 데이터베이스에서 특정 정보를 조회하는 과정처럼 보일 수 있지만,  ChatGPT는 학습된 데이터를 기반으로 통계적 언어 모델을 통해 답변을 생성하는 것이기 때문에 새로운 문장을 만들어내는 생성형 AI입니다. 즉, 답변은 기존 정보를 기반으로 하지만, 완전히 동일한 답변을 "찾아내는" 것이 아니라 학습된 패턴을 이용해 "새롭게 생성"된 것입니다.

# 여기에 대한 의문점? 그러면 ML이 아닌데?
> 생성 능력 부족: 질문이 특정 chunk와 직접적으로 매칭되지 않을 경우, 답변이 부정확하거나 누락될 수 있습니다.
예를 들어, 여러 chunk에 걸친 질문에 대해 통합된 답변을 제공하기 어렵습니다.
의미 해석의 한계:
질문과 chunk 간의 연관성이 단순 유사도 계산에 의존하므로, 의미적으로 연결된 내용을 잘못 해석할 가능성이 있습니다.
응답의 다양성 부족: 생성형 모델은 동일한 질문에도 다양한 표현으로 답을 생성할 수 있지만, chunk 기반 방식은 주어진 텍스트의 범위를 벗어나지 않습니다.

- 헷갈려서는 안된다. 검색 시스템이 아니라 생성형 모델을 만드는 것이다.

# Fine-tuned LLM의 접근 방식 (chat 선생)
- End-to-End 처리
    - 질문이 주어지면 모델이 데이터를 기반으로 직접적으로 답변을 생성합니다.모델 내부에는 질문과 답변의 패턴, 그리고 데이터를 통해 학습한 관계가 내재되어 있습니다.
- 문맥 기반 답변
    - Fine-tuning 과정에서 학습한 내용을 바탕으로 문맥적, 논리적 연결을 고려하여 답변을 생성합니다. 입력 데이터(설교집)의 구조와 내용이 잘  반영되어야 합니다.
- 특징적인 차별화
    - Fine-tuned LLM은 질문과 데이터의 직접적인 일치를 필요로 하지 않습니다. 대신, 학습된 표현과 패턴을 활용해 답을 추론합니다.이를 통해, 학습 데이터에 없는 질문도 어느 정도 일반화된 답변을 생성할 수 있습니다.

# 결론
- 혼란하다. chunk를 나눠야 할지 말아야 할지.
- chunk를 나누지 않고 질문-답변 쌍 또는 문맥 기반 학습 데이터를 넣으면 학습이 된다는데 그 쌍을 얼마나 넣어줘야 할 것인가?
- 또한 RGA(Retrieval-Augmented Generation) 등의 기법을 사용하려면 chunk가 필수이다.
- 일단 유사도 검색이 들어가면 모양이 좀 빠지는 느낌이 없지 않지만 우선 특화된 domain에 대한 기본적인 답변은 가능하다.
- 그 후에 유사도 높은 chunk 들을 기반으로 gpt에 던져서 잘 만들어 달라고 하면 품질이 좋아지지 않을까? 정도로 결론을 낸다.  